In [2]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
import os
import shutil
from pathlib import Path
import os.path
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

In [3]:
dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Downloads/nightowls_validation.json'

In [4]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


In [5]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
nms=[cat['name'] for cat in cats]

In [6]:
catIds = coco.getCatIds()

In [9]:
#For viewing in the debugger
#Nice because no prints needed
imgIds = coco.getImgIds()
imgs = coco.loadImgs(ids = imgIds)
annIds = coco.getAnnIds(imgIds=imgIds)
anns = coco.loadAnns(ids = annIds)

In [17]:
#region
def remove_negative_bbox_anns(anns):
    valid_anns = []
    negative_bbox = False
    for ann in anns:
        for bbox_val in ann['bbox']:
            if bbox_val <= 0:
                negative_bbox = True
        if negative_bbox == False:
            valid_anns.append(ann)
            #print(ann['bbox'])
        negative_bbox = False
    #for v in valid_anns:
        #print(v['bbox'])
    return valid_anns

#endregion
"""
getImgIds() -> list[imgId] 
loadImgs(ids : [imgId]) -> list[img]
getAnnIds(imgIds: list[imgId]) -> list[annId]
loadAnns(ids: list[annId]) -> list[ann]

*One imgId can be multiple annIds
*One annId is one ann


#imgIds need to have annotations and those annotations bboxes must not be negative
#write_label_files need
    #filename, which are in imgs
    #catIds, which are in anns
    #bboxes, which are in anns
"""

def get_img_ids(numImgs):
    imgIds = coco.getImgIds()
    imgIds0 = []
    for imgId in imgIds:
        if imgId_has_annotation(imgId) and imgId_has_positive_bbox(imgId):
            imgIds0.append(imgId)
        if(len(imgIds0) == numImgs):
            break
    return imgIds0

def imgId_has_annotation(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    if(len(annIds) > 0):
        return True
    else:
        return False

def imgId_has_positive_bbox(imgId):
    annIds = coco.getAnnIds(imgIds=imgId)
    for ann in anns:
        for bbox_val in ann['bbox']:
            if bbox_val <= 0:
                negative_bbox = True
            else:
                negative_bbox = False 
        if negative_bbox == True:
            return False
        else:
            return True


imgIds = get_img_ids(120)
print(len(imgIds))
#region
#assert len(imgIds) == 120

def get_imgIds_that_have_anns():
    imgIdsPre = coco.getImgIds()
    imgIds = []
    random.shuffle(imgIds)
    i=0
    for imgId in imgIdsPre:
        annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
        if(len(annIds) > 0):
            anns = coco.loadAnns(ids = annIds, catIds=catIds)
            imgIds.append(imgId)
            i += 1
        if i >= 120:
            break
    
    return imgIds

def test_pipeline():
    imgIds = get_imgIds_that_have_anns()
    allAnnIds = []
    for imgId in imgIds:
        annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
        allAnnIds.append(annIds)
        #print(len(coco.getAnnIds(imgIds = imgId, catIds=catIds)))
        assert len(annIds) > 0
    print('All imgIds have annIds')
    allAnns = []
    for annId in allAnnIds:
        anns = coco.loadAnns(annId)
        allAnns.append(anns)
        assert len(anns) > 0
        for ann in anns:
            assert 'bbox' in ann
    print('All annIds have anns')
    print('All anns have bbox')
    print(f'len(imgIds): {len(imgIds)}')
    print(f'len(allAnnIds): {len(allAnnIds)}')
    print(f'len(allAnns): {len(allAnns)}')
def write_label_files(numImgs):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    imgIds = get_imgIds_that_have_anns()

    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            anns = coco.loadAnns(annIds)
            print(len(anns))
            print('-----------------------------')
            anns = remove_negative_bbox_anns(anns)
            for ann in anns:
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                file.write(f'{cat_map[ann["category_id"]]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
    return
def copy_images_to_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        #print(src)
        #print(dest)
        #print('----------------')
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
            #/Users/azakaria/Downloads/nightowls_validation.png is not a file
            #print(f'{src} is a file')
        else:
            print(f'{src} is not a file')
    return

#write_label_files(120) #dataset/labels
#copy_images_to_dataset_dir(Path('/Users/azakaria/Downloads/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/'), '.png') #dataset/images

#endregion

[7000000, 7000001, 7000002, 7000003, 7000004, 7000005, 7000006, 7000007, 7000008, 7000009, 7000010, 7000011, 7000012, 7000013, 7000014, 7000015, 7000016, 7000017, 7000018, 7000019, 7000020, 7000021, 7000022, 7000023, 7000024, 7000025, 7000026, 7000027, 7000028, 7000029, 7000030, 7000031, 7000032, 7000033, 7000034, 7000035, 7000036, 7000037, 7000038, 7000039, 7000040, 7000041, 7000042, 7000043, 7000044, 7000045, 7000046, 7000047, 7000048, 7000049, 7000050, 7000051, 7000052, 7000053, 7000054, 7000055, 7000056, 7000057, 7000058, 7000059, 7000060, 7000114, 7000115, 7000116, 7000117, 7000118, 7000119, 7000120, 7000121, 7000122, 7000123, 7000124, 7000125, 7000126, 7000127, 7000128, 7000129, 7000130, 7000131, 7000132, 7000133, 7000134, 7000135, 7000136, 7000137, 7000138, 7000139, 7000140, 7000141, 7000142, 7000143, 7000144, 7000145, 7000146, 7000147, 7000148, 7000149, 7000150, 7000151, 7000152, 7000153, 7000154, 7000155, 7000156, 7000157, 7000158, 7000159, 7000160, 7000161, 7000162, 7000163, 

In [2]:
print(len(allAnns))
for anns in allAnns:
    for ann in anns:
        print(ann['bbox'])

NameError: name 'allAnns' is not defined

In [46]:
#rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val
#ls nightowls/dataset/images | wc -l ; ls nightowls/dataset/labels | wc -l

/Users/azakaria/Downloads/nightowls_validation.png is not a file
